In [ ]:
try:
    from google.colab import drive
    COLAB_NOTEBOOK = True
except:
    COLAB_NOTEBOOK = False
if COLAB_NOTEBOOK:
    drive.mount("/content/gdrive", force_remount=True)
import tensorflow as tf
import h5py
import random
import time

In [ ]:
# drive.flush_and_unmount()

In [ ]:
# Install sonnet (it doesn't come preinstalled in the colab VM).
if COLAB_NOTEBOOK:
    !pip install -q dm-sonnet

!grep Model: /proc/driver/nvidia/gpus/*/information | awk '{$1="";print$0}'


In [ ]:
# import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
# import h5py
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm
import time
import sonnet as snt
from tqdm import tqdm

print("TensorFlow version: {}".format(tf.__version__))
print("    Sonnet version: {}".format(snt.__version__))
print("    Numpy  version: {}".format(np.__version__))
# tf.config.list_physical_devices('GPU')

In [ ]:
class ConvMaxPool(snt.Module):
    def __init__(self, name=None):
        super(ConvMaxPool, self).__init__(name=name)
        self.conv = snt.Conv1D(output_channels=64, kernel_shape=64, stride=1, rate=1,
                               padding="SAME", with_bias=True,
                               data_format="NWC", name="conv_1d_64"
                               )

    def __call__(self, inputs):
        features = self.conv(inputs)
        outputs = tf.nn.max_pool1d(features, ksize=2, strides=2, padding="VALID", 
                                   data_format='NWC', name="pool_64×1024") #64×1024
        return outputs


class CNN(snt.Module):
    def __init__(self, epsilon, name=None):
        super(CNN, self).__init__(name=name)
        self.layers = [
            ConvMaxPool(name="conv"+str(ii))
            for ii in range(6)
        ]
        self.conv_in = snt.Conv1D(output_channels=64, kernel_shape=2, stride=1, rate=1,
                       padding="SAME", with_bias=True,
                       data_format="NWC", name="conv_1d_64"
                       )

        self.linear = snt.Linear(output_size=128, with_bias=True)
        self.linear1 = snt.Linear(output_size=128, with_bias=True)
        self.linear2 = snt.Linear(output_size=24, with_bias=True)


    # @snt.once
    def __initialize(self, weight):
        pass

    def __call__(self, inputs):
        features = self.conv_in(inputs) #2×1024
        features = tf.nn.max_pool1d(features, ksize=2, strides=2, padding="VALID", 
                                    data_format='NWC', name="pool_64×1024") #64×1024
        for layer in self.layers:
            features = layer(features)    
        features = snt.flatten(features)

        features = self.linear(features)
        features = tf.nn.relu(features)
        features = self.linear1(features)
        features = tf.nn.relu(features)

        features = self.linear2(features)
        output = tf.nn.softmax(features)
        return output


def step(batch, labels):
    with tf.GradientTape() as tape:
        logits = radio_model(batch)
        loss = cross_entropy_loss(logits, labels)

    params = radio_model.trainable_variables
    grads = tape.gradient(loss, params)
    optimizer.apply_gradients(zip(grads, params))
    return loss

def cross_entropy_loss(logits, target):
    return -tf.reduce_sum(target*tf.math.log(logits))

optimizer = tf.keras.optimizers.Adam()

In [ ]:
radio_model = CNN(epsilon=1e-8, name="CNN")
print(radio_model)

In [ ]:
class radioML:
    def __init__(self, file, batch_size):
        self.file = file
        self.batch_size = batch_size

    def __call__(self):
        with h5py.File(self.file, 'r') as f:
            datasize = len(f['X'])
            while True:
                idxs = random.sample(range(datasize), self.batch_size)
                yield (f['X'][sorted(idxs)], f['Y'][sorted(idxs)])
                # f.close()


In [ ]:
if COLAB_NOTEBOOK:
    hdf5_path = '/content/gdrive/My Drive/data/GOLD_XYZ.hdf5'
else:      
    hdf5_path = '/media/bognev/CE50072F50071DB9/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5'
    
batch_size = 256
num_batches = 100
num_epochs = 10

radio_data = tf.data.Dataset.from_generator(radioML(hdf5_path, 1), 
                output_types=(tf.float32, tf.float32),
                output_shapes=(tf.TensorShape([1, 1024, 2]), tf.TensorShape([1, 24]))) #2555904, 1024, 2]

loss_history = []
loss = 0
i=0
start_time = time.perf_counter()
for radio_batch in radio_data.take(batch_size*num_batches).batch(batch_size).prefetch(512).repeat(num_epochs):
    batch, labels = radio_batch
#     loss = step(batch, labels)
#     loss_history.append(loss.numpy().mean())
    i+=1
tf.print("Execution time:", time.perf_counter() - start_time)         
print("num_batches: ", i, tf.reshape(batch, [batch_size, 1024, 2]).shape)    
